## ComfyUI Runner for Google Colab (Versão Final)

Este notebook configura e executa o ComfyUI no Google Colab, utilizando modelos já sincronizados do seu Google Drive.

In [ ]:
from IPython.display import display, HTML, Javascript

display(HTML('''
<div style="text-align: center; padding: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 15px; margin: 20px 0;">
  <h1 style="color: white; margin: 0 0 20px 0;">🚀 Colab Auto-Start System</h1>
  <button id="startBtn" onclick="startAll()"
          style="padding: 20px 50px; font-size: 24px; background: white;
                 border: none; border-radius: 10px; cursor: pointer;
                 font-weight: bold; color: #667eea; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    ▶️ INICIAR TUDO
  </button>
  <p style="color: white; margin-top: 15px; font-size: 14px;">
    Clique para executar todas as células automaticamente
  </p>
</div>

<script>
function startAll() {
  document.getElementById('startBtn').disabled = true;
  document.getElementById('startBtn').innerHTML = '⏳ Executando...';

  // Executar todas as células
  const cells = Jupyter.notebook.get_cells();
  for (let i = 0; i < cells.length; i++) {
    if (i > 0) { // Pula a primeira célula (este botão)
      cells[i].execute();
    }
  }

  setTimeout(() => {
    document.getElementById('startBtn').innerHTML = '✅ Executando!';
  }, 1000);
}
</script>
'''))

In [1]:
import os

# COLE SEUS VALORES AQUI:

from google.colab import userdata
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
COMFYUI_URL_GIST_ID = userdata.get('COMFYUI_URL_GIST_ID')


print("✅ Secrets configuradas! COMFYUI_URL_GIST_ID, GITHUB_TOKEN")

✅ Secrets configuradas! COMFYUI_URL_GIST_ID, GITHUB_TOKEN


## Versão 1 - ComfyUI


In [2]:

"""
FLUXO:
  Você abre Colab (1 clique)
    ↓
  Notebook executa automaticamente
    ↓
  ComfyUI inicia
    ↓
  Este código:
    - Publica URL no Gist
    - Dispara GitHub Actions automaticamente
    - Monitora inatividade
    - Auto-shutdown após timeout
"""

import time
import threading
import requests
import json
import os
from datetime import datetime, timedelta
from IPython.display import display, HTML
from google.colab import userdata





# ════════════════════════════════════════════════════════════════════════════
# CONFIGURAÇÃO - AJUSTE AQUI!
# ════════════════════════════════════════════════════════════════════════════

# Timeouts e configurações
TIMEOUT_MINUTES = 30  # Auto-shutdown após X minutos de inatividade
                      # 30 = muito econômico
                      # 60 = balanceado
                      # 90 = mais flexível

# GitHub configurações
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_REPO = "Patricia7sp/AI_film"       # Seu repositório

# Gist configurações
COMFYUI_URL_GIST_ID  # ID do seu Gist

# ComfyUI URL (será obtida automaticamente ou você pode definir)
COMFYUI_URL = os.getenv("COMFYUI_URL")  # ou definir manualmente após obter


# ════════════════════════════════════════════════════════════════════════════
# CLASSE PRINCIPAL - GERENCIA TUDO
# ════════════════════════════════════════════════════════════════════════════

class ColabCompleteManager:
    """
    Gerenciador completo do Colab:
    - Monitor de inatividade
    - Trigger do GitHub Actions
    - Atualização do Gist
    - Status em tempo real
    """

    def __init__(self, timeout_minutes=30, github_token=None, github_repo=None,
                 gist_id=None, comfyui_url=None):
        # Configurações
        self.timeout = timeout_minutes * 60
        self.timeout_minutes = timeout_minutes
        self.github_token = github_token
        self.github_repo = github_repo
        self.gist_id = gist_id
        self.comfyui_url = comfyui_url

        # Estado
        self.last_activity = time.time()
        self.start_time = time.time()
        self.running = True
        self.total_requests = 0
        self.status = "starting"
        self.github_triggered = False

    # ────────────────────────────────────────────────────────────────────────
    # GIST - ATUALIZAÇÃO DE STATUS
    # ────────────────────────────────────────────────────────────────────────

    def update_gist(self, status=None):
        """Atualiza Gist com status atual"""
        if not self.gist_id or not self.github_token:
            return False

        try:
            status_data = {
                "comfyui_url": self.comfyui_url,
                "status": status or self.status,
                "last_activity": datetime.now().isoformat(),
                "uptime_minutes": int(self.get_uptime()),
                "inactive_minutes": int(self.get_inactive_time()),
                "auto_shutdown_in_minutes": int(self.get_time_until_shutdown()),
                "total_requests": self.total_requests,
                "timeout_minutes": self.timeout_minutes,
                "github_actions_triggered": self.github_triggered
            }

            url = f"https://api.github.com/gists/{self.gist_id}"
            headers = {
                "Authorization": f"token {self.github_token}",
                "Accept": "application/vnd.github.v3+json"
            }

            # Obter conteúdo atual
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                gist_data = response.json()
                filename = list(gist_data['files'].keys())[0]

                # Atualizar
                files = {
                    filename: {
                        "content": json.dumps(status_data, indent=2)
                    }
                }

                response = requests.patch(url, headers=headers, json={"files": files})
                return response.status_code == 200

        except Exception as e:
            print(f"⚠️ Erro ao atualizar Gist: {e}")
            return False

    # ────────────────────────────────────────────────────────────────────────
    # GITHUB ACTIONS - TRIGGER AUTOMÁTICO
    # ────────────────────────────────────────────────────────────────────────

    def trigger_github_actions(self):
        """Dispara GitHub Actions automaticamente"""
        if not self.github_token or not self.github_repo:
            print("⚠️ GitHub não configurado, pulando trigger")
            return False

        if self.github_triggered:
            print("✅ GitHub Actions já foi disparado anteriormente")
            return True

        print("=" * 70)
        print("🚀 DISPARANDO GITHUB ACTIONS")
        print("=" * 70)

        payload = {
            "event_type": "colab-ready",
            "client_payload": {
                "comfyui_url": self.comfyui_url,
                "triggered_by": "colab",
                "timestamp": datetime.now().isoformat(),
                "status": "ready"
            }
        }

        url = f"https://api.github.com/repos/{self.github_repo}/dispatches"
        headers = {
            "Accept": "application/vnd.github.v3+json",
            "Authorization": f"token {self.github_token}",
            "Content-Type": "application/json"
        }

        try:
            response = requests.post(url, headers=headers, json=payload, timeout=10)

            if response.status_code == 204:
                print("✅ GitHub Actions disparado com sucesso!")
                print(f"👀 Acompanhe em: https://github.com/{self.github_repo}/actions")
                print("=" * 70)
                self.github_triggered = True
                self.update_gist("active")
                return True
            else:
                print(f"❌ Falha: {response.status_code}")
                return False

        except Exception as e:
            print(f"❌ Erro ao disparar: {e}")
            return False

    # ────────────────────────────────────────────────────────────────────────
    # MONITOR DE INATIVIDADE
    # ────────────────────────────────────────────────────────────────────────

    def get_inactive_time(self):
        """Tempo de inatividade em minutos"""
        return (time.time() - self.last_activity) / 60

    def get_uptime(self):
        """Tempo total ativo em minutos"""
        return (time.time() - self.start_time) / 60

    def get_time_until_shutdown(self):
        """Tempo restante até shutdown em minutos"""
        return max(0, self.timeout_minutes - self.get_inactive_time())

    def update_activity(self):
        """Atualiza timestamp de atividade"""
        self.last_activity = time.time()
        self.total_requests += 1
        self.status = "active"
        print(f"✅ Atividade! Timer resetado (Req #{self.total_requests})")

    def monitor_inactivity(self):
        """Loop de monitoramento (roda em thread)"""
        print(f"🔍 Monitor iniciado - checando a cada 1 minuto")
        print(f"⏱️ Auto-shutdown: {self.timeout_minutes} min de inatividade")
        print("")

        while self.running:
            time.sleep(60)

            inactive_min = self.get_inactive_time()
            time_remaining = self.timeout_minutes - inactive_min

            # Atualizar Gist a cada 5 minutos
            if int(inactive_min) % 5 == 0:
                self.update_gist()

            # Avisos progressivos
            if time_remaining <= 5 and time_remaining > 4:
                self.status = "idle"
                print(f"⚠️ ATENÇÃO: Auto-shutdown em 5 minutos!")
                self.update_gist("idle")

            elif time_remaining <= 2 and time_remaining > 1:
                print(f"⚠️ Auto-shutdown em 2 minutos!")

            elif time_remaining <= 1 and time_remaining > 0:
                print(f"⚠️ ÚLTIMA CHANCE: 1 minuto!")

            # Shutdown
            if inactive_min >= self.timeout_minutes:
                print("=" * 70)
                print(f"💤 INATIVO por {inactive_min:.1f} minutos")
                print(f"💰 Economizando compute units...")
                print(f"📊 Sessão: {self.get_uptime():.1f}min total, {self.total_requests} requisições")
                print("=" * 70)
                print("🔌 Desconectando em 10 segundos...")

                self.update_gist("offline")
                time.sleep(10)

                try:
                    from google.colab import runtime
                    runtime.unassign()
                except Exception as e:
                    print(f"❌ Erro ao desconectar: {e}")

                break

    # ────────────────────────────────────────────────────────────────────────
    # INICIALIZAÇÃO
    # ────────────────────────────────────────────────────────────────────────

    def start(self, auto_trigger_github=True):
        """Inicia todos os sistemas"""
        print("=" * 70)
        print("🎯 COLAB COMPLETE MANAGER - INICIANDO")
        print("=" * 70)
        print(f"⏱️ Timeout: {self.timeout_minutes} minutos")
        print(f"🔗 ComfyUI: {self.comfyui_url or 'Aguardando...'}")
        print(f"🐙 GitHub: {self.github_repo if self.github_repo else 'Não configurado'}")
        print(f"📝 Gist: {self.gist_id if self.gist_id else 'Não configurado'}")
        print("=" * 70)
        print("")

        # Atualizar Gist inicial
        if self.comfyui_url:
            self.status = "ready"
            self.update_gist("ready")

            # Disparar GitHub Actions automaticamente
            if auto_trigger_github:
                time.sleep(2)  # Pequeno delay para estabilizar
                self.trigger_github_actions()

        # Iniciar monitor em thread
        thread = threading.Thread(target=self.monitor_inactivity)
        thread.daemon = True
        thread.start()

        # UI
        self.display_ui()

        print("✅ Todos os sistemas iniciados!")
        print("")

    def display_ui(self):
        """Interface visual"""
        html = f"""
        <div style="border: 3px solid #4CAF50; border-radius: 10px; padding: 20px;
                    background: linear-gradient(135deg, #f0f8ff 0%, #e6f7ff 100%);
                    font-family: 'Segoe UI', Tahoma, sans-serif; margin: 10px 0;">
            <h2 style="margin: 0 0 15px 0; color: #2196F3; display: flex; align-items: center;">
                🎯 <span style="margin-left: 10px;">Colab Manager Ativo</span>
            </h2>

            <div style="background: white; padding: 15px; border-radius: 5px; margin-bottom: 10px;">
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                    <div>
                        <strong>⏱️ Timeout:</strong> {self.timeout_minutes} minutos<br>
                        <strong>🔗 ComfyUI:</strong> {'✅ Configurado' if self.comfyui_url else '⏳ Aguardando'}<br>
                        <strong>📝 Gist:</strong> {'✅ Ativo' if self.gist_id else '❌ Não configurado'}
                    </div>
                    <div>
                        <strong>🐙 GitHub Actions:</strong> {'✅ Configurado' if self.github_repo else '❌ Não configurado'}<br>
                        <strong>📊 Status:</strong> <span style="color: #4CAF50; font-weight: bold;">{self.status.upper()}</span><br>
                        <strong>🚀 Auto-trigger:</strong> {'✅ Habilitado' if self.github_repo else '❌ Desabilitado'}
                    </div>
                </div>
            </div>

            <div style="background: #fff3cd; padding: 10px; border-radius: 5px; border-left: 4px solid #ffc107;">
                <strong>💡 Como funciona:</strong><br>
                • Sistema monitora inatividade automaticamente<br>
                • A cada requisição, timer reseta<br>
                • Após {self.timeout_minutes}min sem uso → auto-shutdown<br>
                • GitHub Actions disparado automaticamente quando pronto<br>
                • Acompanhe pipeline em tempo real no GitHub!
            </div>
        </div>
        """
        display(HTML(html))


# ════════════════════════════════════════════════════════════════════════════
# INICIALIZAÇÃO AUTOMÁTICA
# ════════════════════════════════════════════════════════════════════════════

# Criar gerenciador
manager = ColabCompleteManager(
    timeout_minutes=TIMEOUT_MINUTES,
    github_token=GITHUB_TOKEN,
    github_repo=GITHUB_REPO,
    gist_id=COMFYUI_URL_GIST_ID,
    comfyui_url=COMFYUI_URL
)

# Iniciar sistemas
manager.start(auto_trigger_github=True)

print("🎉 Setup completo! Tudo está automatizado!")
print("")
print("📋 O que acontece agora:")
print("  1. Sistema monitora inatividade")
print(f"  2. Auto-shutdown após {TIMEOUT_MINUTES}min sem uso")
print("  3. GitHub Actions será disparado automaticamente")
print("  4. Você acompanha tudo no GitHub!")
print("")
print(f"👀 Acompanhe: https://github.com/{GITHUB_REPO}/actions")
print("")


# ════════════════════════════════════════════════════════════════════════════
# FUNÇÕES DE CONVENIÊNCIA
# ════════════════════════════════════════════════════════════════════════════

def set_comfyui_url(url):
    """Define URL do ComfyUI e dispara GitHub Actions"""
    manager.comfyui_url = url
    manager.update_gist("ready")
    manager.trigger_github_actions()
    print(f"✅ ComfyUI URL configurada: {url}")
    print("🚀 GitHub Actions disparado!")


def keep_alive():
    """Mantém sistema ativo (reseta timer)"""
    manager.update_activity()


def force_shutdown():
    """Força shutdown imediato"""
    manager.running = False
    print("🔌 Shutdown forçado!")


# Exportar manager para uso global
__manager = manager


🎯 COLAB COMPLETE MANAGER - INICIANDO
⏱️ Timeout: 30 minutos
🔗 ComfyUI: Aguardando...
🐙 GitHub: Patricia7sp/AI_film
📝 Gist: 2ba46d9b8484d4e13a292bde736e71ed

🔍 Monitor iniciado - checando a cada 1 minuto
⏱️ Auto-shutdown: 30 min de inatividade



✅ Todos os sistemas iniciados!

🎉 Setup completo! Tudo está automatizado!

📋 O que acontece agora:
  1. Sistema monitora inatividade
  2. Auto-shutdown após 30min sem uso
  3. GitHub Actions será disparado automaticamente
  4. Você acompanha tudo no GitHub!

👀 Acompanhe: https://github.com/Patricia7sp/AI_film/actions



# CÉLULA 1: Preparar Ambiente e Clonar ComfyUI
=============================================

In [3]:
!echo "⏳ Preparando ambiente e instalando dependências compatíveis..."

# Instalar aria2 para downloads mais rápidos
!apt-get update -qq
!apt-get install -y -qq aria2

# Reinstalar PyTorch com versões compatíveis para Colab
!echo "🔧 Instalando PyTorch compatível..."
!pip uninstall -y torch torchvision torchaudio --quiet
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 --quiet

# Clonar o repositório ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
%cd /content/ComfyUI

!echo "✅ Repositório ComfyUI clonado."

# Instalar dependências do ComfyUI (exceto PyTorch que já instalamos)
!echo "⏳ Instalando dependências do ComfyUI..."
!pip install -r requirements.txt --quiet --no-deps
!pip install Pillow numpy opencv-python transformers accelerate --quiet

!echo "✅ Dependências instaladas com versões compatíveis."

⏳ Preparando ambiente e instalando dependências compatíveis...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers

In [4]:
# Célula do Google Colab para Setup e Início do ComfyUI - VERSÃO CORRIGIDA

# 1. Navegar para o diretório base '/content/'
%cd /content/

# 2. Verificar se o ComfyUI já foi clonado
import os

comfyui_dir = "/content/ComfyUI"

if os.path.exists(comfyui_dir):
    print(f"Diretório {comfyui_dir} já existe.")
    %cd {comfyui_dir}
    print("Atualizando o ComfyUI com 'git pull'...")
    !git pull
else:
    print(f"Clonando o repositório ComfyUI para {comfyui_dir}...")
    !git clone https://github.com/comfyanonymous/ComfyUI.git {comfyui_dir}
    %cd {comfyui_dir}

# 3. Instalar as dependências do ComfyUI com correções
print("Atualizando pip...")
!pip install --upgrade pip

# Forçar uma versão compatível do NumPy ANTES de instalar o resto
print("Instalando uma versão compatível do NumPy (numpy<2)...")
!pip install "numpy~=1.26.4"

# Instalar as dependências principais do ComfyUI
print("Instalando dependências do ComfyUI a partir de requirements.txt...")
!pip install -r requirements.txt

# Instalar explicitamente torchsde para garantir que 'trampoline' seja instalado
print("Instalando/Verificando torchsde e sua dependência 'trampoline'...")
!pip install torchsde

# Instalar uma versão compatível de safetensors
print("Instalando versão compatível de safetensors...")
!pip uninstall -y safetensors
!pip install safetensors==0.3.1

# Install a compatible version of transformers
print("Instalando versão compatível de transformers...")
!pip uninstall -y transformers
!pip install transformers==4.37.2

# 4. Iniciar o ComfyUI
print("Iniciando ComfyUI...")
!python main.py --listen --port 8188 --preview-method auto

/content
Diretório /content/ComfyUI já existe.
/content/ComfyUI
Atualizando o ComfyUI com 'git pull'...
Already up to date.
Atualizando pip...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.
Instalando uma versão compatível do NumPy (numpy<2)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.0 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that a

Instalando dependências do ComfyUI a partir de requirements.txt...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 MB 74.1 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 77.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 MB 140.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 60.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 122.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [comfyui-workflow-templates-core]
Instalando/Verificando torchsde e sua dependência 'trampoline'...
Instalando versão compatível de safetensors...
Found existing installation: safetensors 0.7.0
Uninstalling safetensors-0.7.0:
  Successfully uninstalled safetensors-0.7.0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for safetensors (pyproject.toml) did not

# CÉLULA 2: Montar Google Drive
============================

In [1]:
from google.colab import drive

!echo "⏳ Montando Google Drive... (Siga as instruções para autorizar)"
drive.mount('/content/drive')
!echo "✅ Google Drive montado em /content/drive"

⏳ Montando Google Drive... (Siga as instruções para autorizar)
Mounted at /content/drive
✅ Google Drive montado em /content/drive


# CÉLULA 2.5: Detectar Estrutura Existente de Modelos
====================================================

In [2]:
import os

print("🔍 Procurando estrutura existente de modelos no Google Drive...")

# Possíveis localizações dos modelos no Google Drive
possible_base_paths = [
    "/content/drive/MyDrive/comfy_models",
    "/content/drive/MyDrive/ComfyUI_models",
    "/content/drive/MyDrive/models",
    "/content/drive/MyDrive/ComfyUI/models",
    "/content/drive/MyDrive/ComfyUI"
]

# Diretórios de modelos que procuramos
model_subdirs = ["checkpoints", "vae", "loras", "controlnet", "embeddings", "upscale_models", "clip_vision", "gligen"]

GDRIVE_MODELS_PATH = None
found_models = {}

print("📁 Verificando possíveis localizações...")
for base_path in possible_base_paths:
    print(f"   Verificando: {base_path}")
    if os.path.exists(base_path):
        print(f"   ✅ Diretório existe: {base_path}")

        # Verificar quantos subdiretórios de modelos existem aqui
        found_count = 0
        temp_found = {}

        for subdir in model_subdirs:
            full_path = os.path.join(base_path, subdir)
            if os.path.exists(full_path):
                found_count += 1
                temp_found[subdir] = full_path
                print(f"      ✅ Encontrado: {subdir} ({len(os.listdir(full_path))} arquivos)")
            else:
                print(f"      ❌ Não encontrado: {subdir}")

        # Se encontrou pelo menos alguns diretórios de modelos, usar este caminho
        if found_count > 0:
            GDRIVE_MODELS_PATH = base_path
            found_models = temp_found
            print(f"   🎯 Usando este diretório base: {base_path} ({found_count} subdiretórios encontrados)")
            break
    else:
        print(f"   ❌ Não existe: {base_path}")

if not GDRIVE_MODELS_PATH:
    # Se não encontrou nada, criar estrutura padrão
    GDRIVE_MODELS_PATH = "/content/drive/MyDrive/comfy_models"
    print(f"\n⚠️ Nenhuma estrutura existente encontrada.")
    print(f"📁 Criando nova estrutura em: {GDRIVE_MODELS_PATH}")

    for subdir in model_subdirs:
        full_path = os.path.join(GDRIVE_MODELS_PATH, subdir)
        os.makedirs(full_path, exist_ok=True)
        found_models[subdir] = full_path
        print(f"   ✅ Criado: {subdir}")
else:
    print(f"\n✅ Estrutura existente detectada em: {GDRIVE_MODELS_PATH}")

    # Criar diretórios que não existem
    for subdir in model_subdirs:
        if subdir not in found_models:
            full_path = os.path.join(GDRIVE_MODELS_PATH, subdir)
            os.makedirs(full_path, exist_ok=True)
            found_models[subdir] = full_path
            print(f"   📁 Criado diretório faltante: {subdir}")

# Salvar informações para próximas células
with open('/tmp/gdrive_models_path.txt', 'w') as f:
    f.write(GDRIVE_MODELS_PATH)

print(f"\n🎯 Diretório base final: {GDRIVE_MODELS_PATH}")
print("📋 Resumo dos diretórios de modelos:")
for subdir, path in found_models.items():
    file_count = len(os.listdir(path)) if os.path.exists(path) else 0
    print(f"   {subdir}: {file_count} arquivos")

🔍 Procurando estrutura existente de modelos no Google Drive...
📁 Verificando possíveis localizações...
   Verificando: /content/drive/MyDrive/comfy_models
   ✅ Diretório existe: /content/drive/MyDrive/comfy_models
      ✅ Encontrado: checkpoints (8 arquivos)
      ✅ Encontrado: vae (1 arquivos)
      ✅ Encontrado: loras (3 arquivos)
      ✅ Encontrado: controlnet (1 arquivos)
      ✅ Encontrado: embeddings (1 arquivos)
      ✅ Encontrado: upscale_models (1 arquivos)
      ✅ Encontrado: clip_vision (1 arquivos)
      ✅ Encontrado: gligen (1 arquivos)
   🎯 Usando este diretório base: /content/drive/MyDrive/comfy_models (8 subdiretórios encontrados)

✅ Estrutura existente detectada em: /content/drive/MyDrive/comfy_models

🎯 Diretório base final: /content/drive/MyDrive/comfy_models
📋 Resumo dos diretórios de modelos:
   checkpoints: 8 arquivos
   vae: 1 arquivos
   loras: 3 arquivos
   controlnet: 1 arquivos
   embeddings: 1 arquivos
   upscale_models: 1 arquivos
   clip_vision: 1 arquivos

In [ ]:
!chmod +x /content/drive/MyDrive/comfy_models/disney_models/install_disney_models.sh


In [ ]:
# 2) Tornar o script executável (uma vez só)
!/content/drive/MyDrive/comfy_models/disney_models/install_disney_models.sh

# CÉLULA 3: Criar Links Simbólicos para os Modelos
===============================================

In [3]:
import os

print("⏳ Configurando links simbólicos para os modelos...")

# Ler o caminho detectado da célula anterior
with open('/tmp/gdrive_models_path.txt', 'r') as f:
    GDRIVE_MODELS_PATH = f.read().strip()

print(f"📁 Usando diretório base: {GDRIVE_MODELS_PATH}")

# Diretórios de modelos
model_directories = {
    "checkpoints": "checkpoints",
    "vae": "vae",
    "loras": "loras",
    "controlnet": "controlnet",
    "embeddings": "embeddings",
    "upscale_models": "upscale_models",
    "clip_vision": "clip_vision",
    "disney_models": "disney_models",
    "gligen": "gligen"
}

comfyui_base_models_dir = "/content/ComfyUI/models"

print("\n🔗 Criando links simbólicos...")
for comfy_subdir, gdrive_subdir_name in model_directories.items():
    local_model_path = os.path.join(comfyui_base_models_dir, comfy_subdir)
    gdrive_model_path = os.path.join(GDRIVE_MODELS_PATH, gdrive_subdir_name)

    print(f"\n   📂 {comfy_subdir}:")
    print(f"      Local: {local_model_path}")
    print(f"      GDrive: {gdrive_model_path}")

    # Verificar se o diretório no GDrive existe
    if not os.path.exists(gdrive_model_path):
        print(f"      ⚠️ Diretório no GDrive não existe, criando...")
        os.makedirs(gdrive_model_path, exist_ok=True)
    else:
        file_count = len(os.listdir(gdrive_model_path))
        print(f"      ✅ GDrive OK ({file_count} arquivos)")

    # Remover diretório local se existir e não for link simbólico
    if os.path.exists(local_model_path) and not os.path.islink(local_model_path):
        print(f"      🗑️ Removendo diretório local existente...")
        os.system(f"rm -rf '{local_model_path}'")
    elif os.path.islink(local_model_path):
        print(f"      ✅ Link simbólico já existe, pulando.")
        continue

    # Criar link simbólico
    try:
        os.symlink(gdrive_model_path, local_model_path)
        print(f"      ✅ Link simbólico criado com sucesso!")
    except Exception as e:
        print(f"      ❌ Erro ao criar link: {e}")
        # Fallback: criar diretório local vazio
        os.makedirs(local_model_path, exist_ok=True)
        print(f"      📁 Criado diretório local como fallback")

print("\n✅ Links simbólicos configurados!")
print("\n📋 Estrutura final do ComfyUI:")
!ls -la /content/ComfyUI/models/

⏳ Configurando links simbólicos para os modelos...
📁 Usando diretório base: /content/drive/MyDrive/comfy_models

🔗 Criando links simbólicos...

   📂 checkpoints:
      Local: /content/ComfyUI/models/checkpoints
      GDrive: /content/drive/MyDrive/comfy_models/checkpoints
      ✅ GDrive OK (8 arquivos)
      🗑️ Removendo diretório local existente...
      ✅ Link simbólico criado com sucesso!

   📂 vae:
      Local: /content/ComfyUI/models/vae
      GDrive: /content/drive/MyDrive/comfy_models/vae
      ✅ GDrive OK (1 arquivos)
      🗑️ Removendo diretório local existente...
      ✅ Link simbólico criado com sucesso!

   📂 loras:
      Local: /content/ComfyUI/models/loras
      GDrive: /content/drive/MyDrive/comfy_models/loras
      ✅ GDrive OK (3 arquivos)
      🗑️ Removendo diretório local existente...
      ✅ Link simbólico criado com sucesso!

   📂 controlnet:
      Local: /content/ComfyUI/models/controlnet
      GDrive: /content/drive/MyDrive/comfy_models/controlnet
      ✅ GDrive O

# CÉLULA 4: Iniciar ComfyUI e Expor Publicamente
=============================================

## Célula 1: Iniciar ComfyUI (e manter rodando)

In [ ]:
%cd /content/ComfyUI
!python main.py --listen --port 8188 --preview-method auto

## Cloudflared

In [ ]:
# Iniciar ComfyUI em background
!cd /content/ComfyUI && nohup python main.py --listen --port 8188 --preview-method auto > comfyui.log 2>&1 &

## CÉLULA 4.5: Expor ComfyUI com Cloudflared (Túnel HTTP seguro)
============================================================================

Esta célula instala e executa o cloudflared para expor o ComfyUI de forma confiável para automação externa.

- O link gerado será do tipo https://xxxx.trycloudflare.com
- Use esse link no seu .env local como COMFYUI_HOST para automação.

In [4]:
# ════════════════════════════════════════════════════════════════════════════
# INICIAR COMFYUI + CLOUDFLARE TUNNEL
# ════════════════════════════════════════════════════════════════════════════

import subprocess
import time
import os
# Import the existing ColabCompleteManager from the previous cell
# Ensure cell _flwFKeHWpU_ is executed before this one
try:
    from __main__ import manager as main_manager
except ImportError:
    main_manager = None
    print("⚠️ Could not import main_manager. Manager updates will be skipped.")


print("🚀 Iniciando ComfyUI...")

# 1. Iniciar ComfyUI em background
subprocess.Popen([
    'python', '/content/ComfyUI/main.py',
    '--listen', '0.0.0.0',
    '--port', '8188'
], stdout=open('/tmp/comfyui.log', 'w'), stderr=subprocess.STDOUT)

print("⏳ Aguardando ComfyUI iniciar (10 seconds)...")
time.sleep(10)

# 1.5. Instalar cloudflared
print("⬇️ Instalando cloudflared...")
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
print("✅ cloudflared instalado.")

print("🌐 Iniciando Cloudflare Tunnel...")

# 2. Iniciar Cloudflare tunnel
subprocess.Popen([
    'cloudflared', 'tunnel',
    '--url', 'http://localhost:8188'
], stdout=open('/tmp/cloudflared.log', 'w'), stderr=subprocess.STDOUT)

print("⏳ Aguardando tunnel criar (5 seconds)...")
time.sleep(5)

# 3. Capturar URL do Cloudflare
import re

# Retry reading the log for a bit in case the URL isn't immediately available
url = None
for i in range(5): # Try up to 5 times
    with open('/tmp/cloudflared.log', 'r') as f:
        log_content = f.read()
        match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', log_content)
        if match:
            url = match.group(0)
            break
    time.sleep(2) # Wait a bit before retrying

if url:
    print(f"✅ ComfyUI iniciado!")
    print(f"🔗 URL: {url}")

    # Definir variável de ambiente para o sistema usar
    os.environ["COMFYUI_URL"] = url

    # Update the manager with the URL and trigger GitHub Actions
    if main_manager:
        try:
            main_manager.comfyui_url = url
            main_manager.update_gist("ready")
            main_manager.trigger_github_actions()
            print("✅ Manager updated and GitHub Actions triggered.")
        except Exception as e:
            print(f"❌ Error updating manager or triggering GitHub Actions: {e}")
    else:
        print("⚠️ Manager object not found. Skipping manager update and GitHub Actions trigger.")

else:
    print("⏳ URL ainda não gerada after multiple attempts.")
    print("📋 Verifique o log: /tmp/cloudflared.log")

⚠️ Could not import main_manager. Manager updates will be skipped.
🚀 Iniciando ComfyUI...
⏳ Aguardando ComfyUI iniciar (10 seconds)...
⬇️ Instalando cloudflared...
--2026-02-15 06:23:28--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64 [following]
--2026-02-15 06:23:29--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/f9298ca8-89c8-41fe-a51f-e24cb2059878?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-15T07%3A00%3A46Z&rscd=attachment%3B+f

#  Versão 2 - ComfyUI

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

In [5]:
"""
🎬 COLAB MANAGER - VERSÃO SIMPLIFICADA (SEM FLASK)

FLUXO ALTERNATIVO:
  1. ComfyUI inicia
  2. URL publicada no Gist
  3. Você cola a história direto no código
  4. GitHub Actions disparado com história
  5. Pipeline executa
"""

import time, requests, json, os
from datetime import datetime
from IPython.display import display, HTML
from google.colab import userdata

# Config
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_REPO = "Patricia7sp/AI_film"
COMFYUI_URL_GIST_ID = userdata.get('COMFYUI_URL_GIST_ID')
COMFYUI_URL = os.getenv("COMFYUI_URL")

# ═══════════════════════════════════════════════════════════════════
# 📝 INSIRA SUA HISTÓRIA AQUI:
# ═══════════════════════════════════════════════════════════════════

STORY = """
O Professor de Matemática

Eu preciso arrumar um jeito de esconder esses gatinhos. Já é a segunda vez que Mia dá cria, mas desta vez ela exagerou e vieram logo oito. Mamãe disse que só suportaria três de cada vez, pois as nossas já são muito levadas e gostam de se enredar nas meadas de lã quando estamos tentando tricotar. Não que eu saiba tricotar. Mas posso segurar a lã para mamãe e ir desembaraçando os fios, enquanto as gatas tentam, por sua vez, caçar os novelos, o que sempre me faz estourar de rir.
	Filhotes de gato são engraçadinhos e todo mundo gosta deles, mas agora, veja bem, vieram oito de uma vez, e até eu mesma sei que oito não é um número razoável. Talvez eu possa escondê-los no jardim e trazer Mia para visitá-los. Talvez possa roubar pão e leite na cozinha da universidade, já que papai é o reitor, até que consiga convencer mamãe a ficar com todos. Senão eles serão doados como da outra vez, o que me fez chorar muito e ficar inconsolável por quase três dias inteiros.
	Minha irmã Lóri tentou me acalmar, deixando que eu tomasse quanto xarope de groselha quisesse, mas acabei ficando enjoada e mais aborrecida ainda.
	- Pare com essa bobagem! – ela me repreendeu, afinal perdendo a paciência – Você já é grande demais para choramingar assim.
Sempre me dizem isso quando choro, mas o curioso é que outras vezes me acham muito pequena. Por exemplo, quando tenho alguma opinião sobre algo que os adultos estão conversando, ouço coisas como "Você não tem tamanho para se meter nesse assunto." Acho que é por viver escutando admoestações desse tipo que fico confusa e às vezes me sinto mesmo tão grande e às vezes tão pequena.
Bem, não vejo nesse jardim lugar algum onde esconder os filhotes e olhe que estou procurando cuidadosamente há mais de dez minutos. O jardim da universidade é imenso, e ainda assim está parecendo minúsculo no momento. Ele deve entender como me sinto.
Há aqui algumas faias bem antigas, de troncos grossos e rugosos. Eu poderia abrigar os bichanos entre suas raízes, mas parece exposto demais se chover. Contudo, tenho medo que se afoguem se eu atravessar para o outro lado e deixá-los muito perto do tanque de peixes.
Eu sei que muita gente acharia que estou tendo trabalho demais para conservar oito simples filhotes, afinal já tenho três gatas, todas fêmeas, e a cada primavera elas podem resolver aparecer com uma nova ninhada. Se eu conseguisse manter todos os gatos por 50 anos e a cada ninhada nascessem 8, então seria 8 vezes 50. Vezes 3, porque são três gatas-mães. Nesse caso eles caberiam todos dentro de casa conosco ou seria preciso que alguém da família se mudasse para um outro lugar? Talvez eu pudesse morar no telhado, se por lá colocassem alguma cobertura no inverno. A conta está errada, tenho que somar ainda 3, que são as gatas originais. De cabeça não vou conseguir calcular, vou pedir a Ludovico que faça isso na hora do chá.
Ludovico é o único que parece entender meu amor pelos animais. Eu não digo só animais que todo mundo gosta, como cavalos, cachorros ou gatos; eu gosto também de lagartas, pássaros e besouros. Agora há pouco, por exemplo, esqueci por completo de que estou procurando um esconderijo, encontrei um formigueiro e me deitei de bruços para acompanhar o caminho das operárias. Poderia ficar assim por horas, elas são muito interessantes. Ludovico uma vez me disse que são capazes de carregar até cem vezes o peso do próprio corpo e ele sabe do que fala porque é professor, mas não de um jeito chato como os outros, e sim de um jeito que fez com que buscássemos uma balança para pesar as formigas e as cargas que carregavam para ver se a teoria estava certa. Infelizmente os insetos não queriam colaborar com a experiência e não paravam quietos, insistindo em escapar pelas laterais do instrumento.
- Estão ocupadas – Ludovico me explicou, e colocou a última formiga delicadamente de volta ao chão.
Meu amigo também gosta de animais e eu posso contar tudo para ele, pois nunca diz que sou grande ou pequena demais, apesar de ele mesmo ser, obviamente, grande no tamanho, com pernas e braços tão compridos.
Fora isso, ele não se parece nadinha com um adulto.
A primeira a simpatizar com ele foi mamãe, quando o conheceu. Então ela pediu a papai que convidasse o professor de matemática da universidade para tomar chá. Lori pôde se sentar à mesa com papai, mamãe e a visita, mas eu e Edith não tivemos permissão, porque desta vez éramos pequenas demais, então ficamos espiando pela porta entreaberta da sala de jantar.
Quando ele chegou, percebi que era alto e magro como um louva-a-Deus, o que me pareceu uma coincidência fantástica, porque mamãe contou que, além de professor de matemática, ele também tinha sido clérigo. Parecia muito tímido e, quando cumprimentou o papai, gaguejou um pouco.
Começaram a conversar e logo Edith se cansou. Apanhou uma boneca e sentou-se ao meu lado, mas, nesse momento, percebi que ela não era a única que estava desinteressada da conversa. O professor-clérigo parecia muito distraído e, quando papai lhe fez uma pergunta, ele foi pego de surpresa e respondeu, assustado:
- O... o q-q-q-que o s-s-senhor disse?
Era tão óbvio que não tinha ouvido nada desde o começo que não consegui segurar uma risada, que saiu com um arquejo, me deixando imediatamente apavorada com a consequência disso. Todo mundo sabe que crianças bem educadas devem ser vistas e não ouvidas, muito menos ouvidas rindo de uma visita ilustre.
Todos naquela mesa se voltaram para mim. Minha irmã se enrijeceu, minha mãe se empertigou e as sobrancelhas do meu pai formaram um V perfeito sobre os olhos. "Vaca, vitela, vime, vespa, viga, voraz", pensei, rapidamente, para tomar coragem, antes de encarar o professor de matemática.
Mas, para minha surpresa, ele parecia feliz com a interrupção. Seus olhos pousados em mim eram de um azul intenso e profundo e ele tinha uma expressão risonha quando exclamou:
- Ah! Existem outras crianças na casa!
- Sim, temos mais duas meninas pequenas. – respondeu mamãe – Mas achamos que elas poderiam incomodar o senhor.
- A pergunta certa é se eu não irei incomodá- las! – e então ele se dirigiu a mim, como ninguém grande fazia: - Eu incomodo você?
Fiz que não com a cabeça, sem conseguir deixar de sorrir. Eu já estava gostando muito dele.
- Venham, pequenas. – condescendeu meu pai – Venham cumprimentar o professor.
Entramos na sala. Fiz uma reverência e Edith tentou repetir aquele sobrenome estranho mas, depois da segunda tentativa, ele a interrompeu, bondosamente:
- Não precisa me chamar assim, Edith. Nem de senhor. Todas três podem me chamar de Ludovico. É como meus amigos me chamam.
Tomamos nossos lugares à mesa. Quando comecei a colocar açúcar no chá, ele olhou para mim, parecendo muito espantado, e perguntou:
- O que é isso?!
- Isso o que?
- Dentro do açucareiro!
Olhei mais de perto, Edith trepou na cadeira e até Lóri se aproximou. Ludovico tirou a tampa, tapou a abertura com a mão e sussurrou:
- Está se mexendo.
O açucareiro trepidou.
Ludovico aproximou o rosto e espiou por entre os dedos.
- Contem até três! – pediu – É um ratinho branco. Vou tirá-lo daí.
- Um! – disse Lóri, tentando espiar também.
- Dois! – eu sussurrei, com a respiração suspensa.
- Três! – gritou Edith, com os olhos arregalados.
E então o professor tirou a mão e algo branco saltou para fora do açucareiro, enquanto nós três gritávamos:
- Um ratinho!!!
E lá estava Ludovico, um alvo guardanapo dobrado nas mãos, piscando alegremente os olhos azuis.
- Faz de novo! – pediu Lóri, sem conseguir se conter.
O professor riu:
- Mágica só se faz uma vez, senão ela perde a graça.
- O senhor sabe fazer outras mágicas? – perguntou Edith, os olhos brilhando.
- Faz mais uma! Faz outra! – nós três implorávamos, na maior algazarra.
Até meus pais estavam rindo.
Aquela foi a primeira demonstração dos vários truques de ilusionismo que Ludovico sabia fazer. Gostaria que ele estivesse aqui agora, porque assim, quem sabe, fizesse surgir um bom esconderijo num passe de mágica. Mas ele está em horário de aula e meu pai proibiu que o perturbemos nessas ocasiões pois, segundo ele, já o obrigamos a ficar conosco por tempo demais. "Principalmente você, mocinha", acrescentou, olhando severamente para mim.
É verdade que muitas tardes se passam com o professor Ludovico. Mas "obrigar" não é um termo justo, pois ele gosta de estar conosco tanto quanto gostamos de estar com ele. Porque ele não só sabe fazer mágicas, como é capaz de criar invenções mirabolantes, que nos mostra quando eu e minhas irmãs vamos visitá-lo. Também gosta de montar quebra cabeças e de jogar cartas. O baralho dele tem uma dama de copas muito esquisita e que me parece um pouco assustadora. Ele diz que não é uma dama, é uma rainha, e que tem essa expressão ameaçadora porque é ela quem manda no reino e até o próprio rei a obedece. Muitas vezes nós alteramos as regras dos jogos. Com o xadrez, por exemplo, eu e ele criamos uma espécie de jogo animado, onde as peças falam e brigam entre si. Quando se vê em perigo, um peão pode montar em um cavalo e fugir a galope. E quando a rainha consegue eliminar uma das peças adversárias, tem que gritar bem alto:
- COOOOOOOORTEM-LHE A CABEEEEEEÇA!!!
Se todo mundo fosse como Ludovico, eu poderia ter quantos bichos quisesse.
E por falar nisso, estou vendo uma toca. Está bem no barranco, um buraco comprido e fundo. Fui avisada muitas vezes para jamais mexer nesses buracos, porque nunca dá para saber a qual animal pertencem. Às vezes são de raposas, mas podem também ser de cobras ou de outros bichos venenosos. Imagino que poderia abrigar os meus gatinhos ali, se o dono fosse um bom anfitrião. Poderia ser um caxinguelê bem velhinho e simpático, que gostasse da ideia de ter netos, e então ele diria: "Venham, meus queridos, vou cuidar de vocês enquanto sua dona Alice convence a mãe dela a não distribui-los por aí. Deitem-se quietinhos e não mexam no meu tricô."
Esse é o tipo de coisa que Ludovico inventaria, porque, além de tudo, ele é também um grande contador de histórias. E elas são sempre engraçadas, um pouco sem pé nem cabeça.
- Não sei como um matemático pode ter tanta imaginação, Dodgson!, ouvi meu pai dizer certa vez ao professor Ludovico, que respondeu, pensativamente:
- Uso a matemática para tentar ordenar um pouco o caos porque a vida, Henry, a vida me parece frequentemente um tanto ou quanto absurda.
Parece que a toca afinal é de um coelho, que estava com pressa demais de entrar nela e nem teve tempo de me dar atenção. Eu poderia me enfiar atrás dele, agora que sei que não é moradia de nenhum animal perigoso, mas que profundidade terá? Não quero cair no centro da Terra.
Acabei de ter uma ideia melhor: vou pedir ao professor Ludovico que cuide dos gatinhos para mim. Ele não irá negar porque sei que sou sua menina Liddle preferida. Vai mantê-los seguros e eu poderei visitá-los todos os dias, até que cresçam. Contarei a ele minha aventura no jardim e explicarei que não achei nenhum lugar adequado para escondê-los.
Preciso resolver essa questão logo porque amanhã o professor Charles Lutwidge Dodgson, vulgo Ludovico e o amigo dele, o senhor Robinson Duckworth (que ele chama de Duck) farão um piquenique seguido de um passeio de barco pelo Tâmisa. Eles pediram permissão a mamãe para levar Lóri, Edith e eu a bordo. Ela disse que poderemos participar da excursão se o tempo estiver firme. O dia hoje foi morno e agradável, creio que amanhã será igual e que nossos planos darão certo. Vou levar tortas de ameixas, quem sabe Ludovico esteja em boa disposição e nos conte uma de suas histórias.

"""

# ═══════════════════════════════════════════════════════════════════

class SimpleColabManager:
    def __init__(self):
        self.github_token = GITHUB_TOKEN
        self.github_repo = GITHUB_REPO
        self.gist_id = COMFYUI_URL_GIST_ID
        self.comfyui_url = COMFYUI_URL
        self.story = STORY.strip()

    def update_gist(self):
        """Atualiza Gist com status"""
        if not self.gist_id or not self.github_token:
            return False

        try:
            status_data = {
                "comfyui_url": self.comfyui_url,
                "status": "ready",
                "timestamp": datetime.now().isoformat(),
                "story_length": len(self.story)
            }

            url = f"https://api.github.com/gists/{self.gist_id}"
            headers = {
                "Authorization": f"token {self.github_token}",
                "Accept": "application/vnd.github.v3+json"
            }

            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                gist_data = response.json()
                filename = list(gist_data['files'].keys())[0]

                files = {
                    filename: {
                        "content": json.dumps(status_data, indent=2)
                    }
                }

                response = requests.patch(url, headers=headers, json={"files": files})
                return response.status_code == 200
        except Exception as e:
            print(f"⚠️ Erro ao atualizar Gist: {e}")
            return False

    def trigger_github(self):
        """Dispara GitHub Actions com história"""
        print("=" * 70)
        print("🚀 DISPARANDO GITHUB ACTIONS")
        print("=" * 70)

        payload = {
            "event_type": "colab-ready",
            "client_payload": {
                "comfyui_url": self.comfyui_url,
                "story": self.story,
                "story_length": len(self.story),
                "timestamp": datetime.now().isoformat()
            }
        }

        url = f"https://api.github.com/repos/{self.github_repo}/dispatches"
        headers = {
            "Authorization": f"token {self.github_token}",
            "Accept": "application/vnd.github.v3+json"
        }

        try:
            res = requests.post(url, headers=headers, json=payload, timeout=10)
            if res.status_code == 204:
                print(f"✅ GitHub Actions disparado!")
                print(f"📖 História: {len(self.story)} caracteres")
                print(f"👀 Acompanhe: https://github.com/{self.github_repo}/actions")
                print("=" * 70)
                return True
            else:
                print(f"❌ Erro: {res.status_code}")
                return False
        except Exception as e:
            print(f"❌ Erro: {e}")
            return False

    def display_ui(self):
        """Exibe interface visual"""
        html = f"""
        <div style="border: 3px solid #4CAF50; border-radius: 10px; padding: 20px;
                    background: linear-gradient(135deg, #f0f8ff 0%, #e6f7ff 100%);
                    font-family: Arial; margin: 10px 0;">
            <h2 style="color: #2196F3;">🎯 Colab Manager - Versão Simples</h2>

            <div style="background: white; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <strong>🔗 ComfyUI:</strong> {self.comfyui_url}<br>
                <strong>📖 História:</strong> {len(self.story)} caracteres<br>
                <strong>🚀 Status:</strong> <span style="color: #4CAF50;">PRONTO</span>
            </div>

            <div style="background: #fff3cd; padding: 10px; border-radius: 5px; border-left: 4px solid #ffc107;">
                <strong>💡 Próximos passos:</strong><br>
                1. ✅ História definida no código<br>
                2. ✅ GitHub Actions será disparado<br>
                3. 👀 Acompanhe no GitHub Actions<br>
                4. 🎬 Pipeline executará e gerará o filme
            </div>
        </div>
        """
        display(HTML(html))

    def start(self):
        print("=" * 70)
        print("🎯 COLAB MANAGER - VERSÃO SIMPLES")
        print("=" * 70)

        # Validações
        if not self.comfyui_url:
            print("❌ COMFYUI_URL não definida!")
            print("💡 Defina: os.environ['COMFYUI_URL'] = 'sua-url'")
            return

        if not self.story:
            print("❌ História não definida!")
            print("💡 Edite a variável STORY no código")
            return

        print(f"🔗 ComfyUI: {self.comfyui_url}")
        print(f"📖 História: {len(self.story)} caracteres")
        print("")

        # Atualizar Gist
        print("📝 Atualizando Gist...")
        self.update_gist()

        # Exibir UI
        self.display_ui()

        # Disparar GitHub Actions
        print("")
        self.trigger_github()

        print("")
        print("=" * 70)
        print("✅ SETUP COMPLETO!")
        print("=" * 70)
        print("")
        print("📋 O que acontece agora:")
        print("  1. GitHub Actions está executando")
        print("  2. Pipeline processará a história")
        print("  3. Imagens serão geradas")
        print("  4. Áudio será criado")
        print("  5. Vídeo final será compilado")
        print("")
        print(f"👀 Acompanhe: https://github.com/{self.github_repo}/actions")

# Executar
manager = SimpleColabManager()
manager.start()


🎯 COLAB MANAGER - VERSÃO SIMPLES
🔗 ComfyUI: https://charge-arcade-focus-changed.trycloudflare.com
📖 História: 11555 caracteres

📝 Atualizando Gist...



🚀 DISPARANDO GITHUB ACTIONS
❌ Erro: 401

✅ SETUP COMPLETO!

📋 O que acontece agora:
  1. GitHub Actions está executando
  2. Pipeline processará a história
  3. Imagens serão geradas
  4. Áudio será criado
  5. Vídeo final será compilado

👀 Acompanhe: https://github.com/Patricia7sp/AI_film/actions


In [ ]:
"""
🎬 COLAB MANAGER COM FLASK UI INTEGRADO

NOVO FLUXO:
  1. ComfyUI inicia
  2. URL publicada no Gist
  3. 🆕 FLASK UI ABRE (pop-up)
  4. Você insere história
  5. História salva
  6. 🆕 DEPOIS dispara GitHub Actions
  7. Pipeline executa
"""

import time, threading, requests, json, os, webbrowser
from datetime import datetime
from IPython.display import display, HTML, Javascript
from google.colab import userdata
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok

# Config
TIMEOUT_MINUTES = 30
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_REPO = "Patricia7sp/AI_film"
COMFYUI_URL_GIST_ID = userdata.get('COMFYUI_URL_GIST_ID')
COMFYUI_URL = os.getenv("COMFYUI_URL")

# Flask App
app = Flask(__name__)
story_data = {"story": None, "submitted": False}

# Configurar ngrok (será iniciado manualmente)
flask_port = 5001  # Usar porta diferente para evitar conflito

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>AI Film - Story Input</title>
    <meta charset="utf-8">
    <style>
        body { font-family: Arial; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
               min-height: 100vh; display: flex; align-items: center; justify-content: center; padding: 20px; }
        .container { background: white; border-radius: 20px; max-width: 800px; padding: 40px; }
        h1 { color: #667eea; }
        textarea { width: 100%; min-height: 200px; padding: 15px; border: 2px solid #e0e0e0;
                   border-radius: 10px; font-size: 16px; }
        button { padding: 15px 30px; border: none; border-radius: 10px; font-size: 1.1em;
                 font-weight: 600; cursor: pointer; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                 color: white; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎬 AI Film - Insira sua História</h1>
        <form id="form">
            <textarea id="story" placeholder="Era uma vez..." required></textarea><br><br>
            <button type="submit">🚀 Enviar e Disparar Pipeline</button>
        </form>
        <div id="status"></div>
    </div>
    <script>
        document.getElementById('form').addEventListener('submit', async (e) => {
            e.preventDefault();
            const story = document.getElementById('story').value;
            const res = await fetch('/submit', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({story})
            });
            const data = await res.json();
            document.getElementById('status').innerHTML = data.success ?
                '✅ Enviado! GitHub Actions disparando...' : '❌ Erro';
        });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/submit', methods=['POST'])
def submit():
    story = request.get_json().get('story', '').strip()
    if story:
        story_data['story'] = story
        story_data['submitted'] = True
        print(f"\n✅ HISTÓRIA RECEBIDA! ({len(story)} chars)\n")
        return jsonify({'success': True})
    return jsonify({'success': False}), 400

# Manager Class
class ColabManager:
    def __init__(self):
        self.github_token = GITHUB_TOKEN
        self.github_repo = GITHUB_REPO
        self.gist_id = COMFYUI_URL_GIST_ID
        self.comfyui_url = COMFYUI_URL
        self.flask_url = None
        self.github_triggered = False

    def start_flask(self):
        """Inicia Flask com ngrok no Colab"""
        print("🌐 Iniciando Flask...")

        # Iniciar ngrok primeiro
        from pyngrok import ngrok

        try:
            # Matar processos anteriores
            ngrok.kill()
        except:
            pass

        # Iniciar túnel ngrok
        try:
            public_url = ngrok.connect(flask_port, bind_tls=True)
            self.flask_url = str(public_url)
            print(f"✅ ngrok túnel criado: {self.flask_url}")
        except Exception as e:
            print(f"❌ Erro ao criar túnel ngrok: {e}")
            return

        # Iniciar Flask em thread
        def run_flask():
            app.run(port=flask_port, use_reloader=False)

        flask_thread = threading.Thread(target=run_flask, daemon=True)
        flask_thread.start()

        # Aguardar Flask iniciar
        time.sleep(3)
        print(f"✅ Flask rodando na porta {flask_port}")

    def open_flask_ui(self):
        if not self.flask_url:
            return
        display(HTML(f'''
        <div style="background: #4CAF50; color: white; padding: 20px; border-radius: 10px; text-align: center;">
            <h2>🎬 Insira sua História</h2>
            <a href="{self.flask_url}" target="_blank"
               style="background: white; color: #4CAF50; padding: 15px 30px; text-decoration: none;
                      border-radius: 5px; font-weight: bold; display: inline-block;">
                📝 ABRIR FLASK UI
            </a>
        </div>
        '''))
        display(Javascript(f'window.open("{self.flask_url}", "_blank");'))

    def wait_for_story(self, timeout_min=10):
        print(f"⏳ Aguardando história... (timeout: {timeout_min}min)")
        start = time.time()
        while time.time() - start < timeout_min * 60:
            if story_data['submitted']:
                print("✅ História recebida!")
                return story_data['story']
            time.sleep(2)
        print("⚠️ Timeout - sem história")
        return None

    def trigger_github(self, story=None):
        if self.github_triggered:
            return True
        print("🚀 Disparando GitHub Actions...")
        payload = {
            "event_type": "colab-ready",
            "client_payload": {
                "comfyui_url": self.comfyui_url,
                "story": story or "",
                "timestamp": datetime.now().isoformat()
            }
        }
        url = f"https://api.github.com/repos/{self.github_repo}/dispatches"
        headers = {"Authorization": f"token {self.github_token}",
                   "Accept": "application/vnd.github.v3+json"}
        res = requests.post(url, headers=headers, json=payload, timeout=10)
        if res.status_code == 204:
            print(f"✅ Disparado! https://github.com/{self.github_repo}/actions")
            self.github_triggered = True
            return True
        return False

    def start(self):
        print("=" * 70)
        print("🎯 INICIANDO COLAB MANAGER")
        print("=" * 70)

        # Verificar se ComfyUI URL está definida
        if not self.comfyui_url:
            print("⚠️ COMFYUI_URL não definida!")
            print("💡 Defina: os.environ['COMFYUI_URL'] = 'sua-url'")
            return

        print(f"🔗 ComfyUI: {self.comfyui_url}")
        print("")

        # Iniciar Flask
        try:
            self.start_flask()
        except Exception as e:
            print(f"❌ Erro ao iniciar Flask: {e}")
            print("💡 Tentando continuar sem Flask UI...")
            self.flask_url = None

        # Abrir UI se Flask iniciou
        if self.flask_url:
            self.open_flask_ui()
            story = self.wait_for_story(10)
        else:
            print("⚠️ Flask UI não disponível")
            print("💡 GitHub Actions será disparado sem história")
            story = None

        # Disparar GitHub Actions
        self.trigger_github(story)

        print("")
        print("=" * 70)
        print("✅ Setup completo!")
        print("=" * 70)

# Executar
manager = ColabManager()
manager.start()


ModuleNotFoundError: No module named 'flask_ngrok'

# ComfyUI backup

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# 🚀 DISPARAR GITHUB ACTIONS MANUALMENTE
# ════════════════════════════════════════════════════════════════════════════

import requests
import json
from google.colab import userdata
from datetime import datetime

# Configurações
COMFYUI_URL = "https://leaving-determined-perfume-realtors.trycloudflare.com"
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
REPO = "Patricia7sp/AI_film"

print("=" * 70)
print("🚀 DISPARANDO GITHUB ACTIONS")
print("=" * 70)
print(f"📦 Repositório: {REPO}")
print(f"🔗 ComfyUI URL: {COMFYUI_URL}")
print("")

# Montar payload
url = f"https://api.github.com/repos/{REPO}/dispatches"
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json",
    "Content-Type": "application/json"
}
payload = {
    "event_type": "colab-ready",
    "client_payload": {
        "comfyui_url": COMFYUI_URL,
        "timestamp": datetime.now().isoformat(),
        "source": "manual_colab_trigger",
        "status": "ready"
    }
}

print("📤 Enviando requisição...")
print(f"   URL: {url}")
print(f"   Event: colab-ready")
print("")

# Fazer requisição
try:
    response = requests.post(url, headers=headers, json=payload, timeout=30)

    print(f"📊 Status Code: {response.status_code}")

    if response.status_code == 204:
        print("")
        print("=" * 70)
        print("✅ SUCESSO! GITHUB ACTIONS DISPARADO!")
        print("=" * 70)
        print("")
        print("👀 Acompanhe o workflow em:")
        print(f"   https://github.com/{REPO}/actions")
        print("")
        print("⏱️ O workflow pode levar 10-30 segundos para aparecer")
        print("🔄 Atualize a página do GitHub Actions para ver")
        print("")
    elif response.status_code == 401:
        print("")
        print("❌ ERRO: Token inválido ou sem permissão")
        print("   Verifique se o token tem scope 'workflow'")
        print("")
    elif response.status_code == 404:
        print("")
        print("❌ ERRO: Repositório não encontrado")
        print(f"   Verifique se '{REPO}' está correto")
        print("")
    else:
        print("")
        print(f"❌ ERRO: {response.status_code}")
        print(f"   Resposta: {response.text}")
        print("")
except Exception as e:
    print("")
    print(f"❌ ERRO na requisição: {e}")
    print("")

print("=" * 70)

In [ ]:
import requests
from google.colab import userdata
from datetime import datetime

COMFYUI_URL = "https://leaving-determined-perfume-realtors.trycloudflare.com"  # ATUALIZAR!
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
REPO = "Patricia7sp/AI_film"

response = requests.post(
    f"https://api.github.com/repos/{REPO}/dispatches",
    headers={"Authorization": f"token {GITHUB_TOKEN}"},
    json={
        "event_type": "colab-ready",
        "client_payload": {"comfyui_url": COMFYUI_URL}
    }
)

print("✅ Agora vai executar DE VERDADE!")
print("👀 https://github.com/Patricia7sp/AI_film/actions")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# DISPARAR GITHUB ACTIONS MANUALMENTE
# ════════════════════════════════════════════════════════════════════════════

# A URL do ComfyUI que você obteve
comfyui_url = "https://velocity-tongue-leader-sharing.trycloudflare.com"

# Chamar a função do manager para disparar
print("🚀 Disparando GitHub Actions...")
print(f"📝 ComfyUI URL: {comfyui_url}")
print("")

# Se você tem o manager global (do código anterior)
try:
    result = _manager.trigger_github_actions(comfyui_url=comfyui_url)
    if result:
        print("✅ GitHub Actions disparado com sucesso!")
        print("👀 Acompanhe em: https://github.com/Patricia7sp/AI_film/actions")
    else:
        print("❌ Erro ao disparar. Veja logs acima.")
except Exception as e:
    print(f"❌ Erro: {e}")
    print("\n💡 Tentando método alternativo...")

    # Método alternativo: chamar API diretamente
    import requests
    from google.colab import userdata

    github_token = userdata.get('GITHUB_TOKEN')
    repo = "Patricia7sp/AI_film"

    url = f"https://api.github.com/repos/{repo}/dispatches"
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }
    payload = {
        "event_type": "colab-ready",
        "client_payload": {
            "comfyui_url": comfyui_url,
            "timestamp": "2025-10-14T00:39:00Z",
            "source": "manual_trigger"
        }
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 204:
        print("✅ GitHub Actions disparado via API!")
        print("👀 Acompanhe em: https://github.com/Patricia7sp/AI_film/actions")
    else:
        print(f"❌ Erro: {response.status_code}")
        print(f"Resposta: {response.text}")

In [ ]:
import requests

# Teste local (dentro do Colab)
try:
    r = requests.get("http://localhost:8188")
    print("Status local:", r.status_code)
except Exception as e:
    print("Erro ao conectar localmente:", e)

In [ ]:
# Teste externo (via cloudflared)
import requests
try:
    r = requests.get("https://barnes-gpl-battery-dates.trycloudflare.com/")
    print("Status cloudflared:", r.status_code)
except Exception as e:
    print("Erro ao conectar via cloudflared:", e)

In [ ]:
#Esta célula pode ajudar a prevenir que o Colab desconecte por inatividade,
# embora o servidor rodando geralmente seja suficiente.
import time
print("🕒 Mantendo o Colab ativo...")
while True:
     time.sleep(300) # Dorme por 5 minutos
     print(f"Ativo às {time.ctime()}")